# INSTALAÇÃO DAS BIBLIOTECAS

In [ ]:
%pip install pyspark

# IMPORTAÇÃO DAS BIBLIOTECAS

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date
import xml.etree.ElementTree as ET

# CRIAÇÃO DA APLICAÇÃO SPARK

In [ ]:
spark = SparkSession.builder.getOrCreate()

# VARIÁVEIS DE APOIO

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
arquivo = '/content/drive/My Drive/ARQ/DADOS_BANCARIOS.xml'

# FUNÇÃO PARA LER ARQUIVO XML

In [ ]:
def ler_xml(arquivo):
    tree = ET.parse(arquivo)
    root = tree.getroot()
    return root

# CRIAÇÃO DE LISTA COM O CONTEÚDO DO XML

In [ ]:
root = ler_xml(arquivo)
dados_bancarios = []

for record in root:
  for record_user_info in record.findall('user_info'):
    name = record_user_info.find('name').text
    address = record_user_info.find('address').text
    gender = record_user_info.find('gender').text
    account_opening_date = record_user_info.find('account_opening_date').text
    customer_type = record_user_info.find('customer_type').text
    date_of_birth = record_user_info.find('date_of_birth').text
  for record_account_info in record.findall('account_info'):
    account_number = record_account_info.find('account_number').text
    account_type = record_account_info.find('account_type').text
    balance = record_account_info.find('balance').text
    currency = record_account_info.find('currency').text
    branch = record_account_info.find('branch').text
  for record_transaction_info in record.findall('transactions/'):
    date = record_transaction_info.find('date').text
    description = record_transaction_info.find('description').text
    amount = record_transaction_info.find('amount').text
    dados_bancarios.append([name, address, gender, account_opening_date, customer_type, date_of_birth, account_number, account_type, balance, currency, branch, date, description, amount])

# LEITURA DA LISTA USANDO SPARK

In [ ]:
df = spark.createDataFrame(dados_bancarios, ['name', 'address', 'gender', 'account_opening_date', 'customer_type', 'date_of_birth', 'account_number', 'account_type', 'balance', 'currency', 'branch', 'date', 'description', 'amount'])

# EXIBINDO UMA AMOSTRA DOS DADOS

In [ ]:
df.show(truncate=False)

+-----------------+--------------------------------------------------+------+--------------------+-------------+-------------+----------------------+------------+--------+--------+------+----------+----------------------------------------------------+------+
|name             |address                                           |gender|account_opening_date|customer_type|date_of_birth|account_number        |account_type|balance |currency|branch|date      |description                                         |amount|
+-----------------+--------------------------------------------------+------+--------------------+-------------+-------------+----------------------+------------+--------+--------+------+----------+----------------------------------------------------+------+
|Justin Sullivan  |95954 Manuel Viaduct\nGarciaberg, OK 85968        |Male  |2018-11-07          |Basic        |1946-08-03   |GB78UOLI44528467272790|Savings     |49774.73|LTL     |63    |2022-07-16|Streamlined tangible appl

# EXIBINDO OS METADADOS (SCHEMA) DO ARQUIVO

In [ ]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- account_opening_date: string (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- account_number: string (nullable = true)
 |-- account_type: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- date: string (nullable = true)
 |-- description: string (nullable = true)
 |-- amount: string (nullable = true)



# AJUSTANDO O SCHEMA DOS DADOS, SE NECESSÁRIO

In [ ]:
df = df.withColumn('amount', df['amount'].cast('double'))
df = df.withColumn('balance ', df['balance'].cast('double'))
df = df.withColumn("account_opening_date", df['account_opening_date'].cast('date'))
df = df.withColumn("date_of_birth", df['date_of_birth'].cast('date'))
df = df.withColumn("date", df['date'].cast('date'))
df = df.withColumn('branch', df['branch'].cast('long'))

df.printSchema()

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- account_opening_date: date (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- date_of_birth: date (nullable = true)
 |-- account_number: string (nullable = true)
 |-- account_type: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- branch: long (nullable = true)
 |-- date: date (nullable = true)
 |-- description: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- balance : double (nullable = true)

